In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pandas_datareader as pdr
from datetime import datetime as dt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import MinMaxScaler

In [2]:
# Load data for specific company
company = 'FB'  # using facebook as an example

start = dt(2012, 1, 1)
end = dt(2021, 8, 1)

data = pdr.DataReader(company, 'yahoo', start, end)

In [3]:
data.head()  # we want to predict the values in the Close column, based on preceding Close column entries

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2012-05-18,45.000000,38.000000,42.049999,38.230000,573576400,38.230000
2012-05-21,36.660000,33.000000,36.529999,34.029999,168192700,34.029999
2012-05-22,33.590000,30.940001,32.610001,31.000000,101786600,31.000000
2012-05-23,32.500000,31.360001,31.370001,32.000000,73600000,32.000000
2012-05-24,33.209999,31.770000,32.950001,33.029999,50237200,33.029999


In [4]:
# Scale data to range between [0,1]
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(data['Close'].values.reshape(-1,1))  # before scaling, reshape the closing values array into a single column vector

In [12]:
prediction_days = 60  # number of days of data to use for making prediction

x_train, y_train = [], []

for x in range(prediction_days, len(scaled_data)):
    x_train.append(scaled_data[x - prediction_days:x])  # prediction_days values before x is the input 
    y_train.append(scaled_data[x, 0])  # the single value after prediction_days values, x, is the output (,0 is to flatten)

# Convert to numpy arrays
x_train, y_train = np.array(x_train), np.array(y_train)

In [15]:
print(x_train[0])
print("-----------")
print(y_train[0])

[[0.05765715]
 [0.04584446]
 [0.03732246]
 [0.040135  ]
 [0.04303192]
 [0.03988187]
 [0.03124737]
 [0.02941921]
 [0.0333849 ]
 [0.02809731]
 [0.02579103]
 [0.02289411]
 [0.0255379 ]
 [0.02413163]
 [0.02635354]
 [0.02610041]
 [0.0271973 ]
 [0.02683167]
 [0.02970047]
 [0.03453804]
 [0.0384756 ]
 [0.03988187]
 [0.03900999]
 [0.039685  ]
 [0.04308817]
 [0.04030376]
 [0.0432288 ]
 [0.04078189]
 [0.03833498]
 [0.03760371]
 [0.03667558]
 [0.03788497]
 [0.03864435]
 [0.03937562]
 [0.04061313]
 [0.03864435]
 [0.03723808]
 [0.03678807]
 [0.03653495]
 [0.02958796]
 [0.02913796]
 [0.03200675]
 [0.03169737]
 [0.03102236]
 [0.03099424]
 [0.03015047]
 [0.03265364]
 [0.0256504 ]
 [0.01681901]
 [0.01524399]
 [0.01119392]
 [0.00885951]
 [0.00649698]
 [0.00945015]
 [0.01178456]
 [0.00840951]
 [0.00840951]
 [0.00922515]
 [0.01147518]
 [0.01088455]]
-----------
0.007453240399990561
